<a href="https://colab.research.google.com/github/sanjaynagi/AnoExpress/blob/main/workflow/notebooks/plot-gene-expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anoexpress -q

![image](https://raw.githubusercontent.com/sanjaynagi/AnoExpressir/main/docs/logo.png) 


In [ ]:
import anoexpress as xpress

In [ ]:
#@title **AnoExpress** { run: "auto" }   
#@markdown This notebook produces interactive strip and boxplots with plotly, to summarise gene expression across 35 *Anopheles* insecticide resistant v susceptible RNA-Sequencing experiments. There are four different analyses to select
#@markdown 'gamb_colu', 'gamb_colu_arab', 'gamb_colu_arab_fun' and 'fun'.  
#@markdown each which integrates a different combination of Anopheles species – ‘gamb_colu’ (An. gambiae and An. coluzzii), ‘gamb_colu_arab’ (adding An. arabiensis), ‘gamb_colu_arab_fun’ (adding An. funestus), and finally ‘fun’ (An. funestus alone).  
#@markdown As you include more species, there are less genes overall in the dataset, as we can only include genes with one-to-many orthologs between species.

analysis = "gamb_colu_arab_fun" #@param ['gamb_colu', 'gamb_colu_arab', 'gamb_colu_arab_fun', 'fun']
GeneID = "AGAP002865" #@param {type:"string"} 
plot_type = "strip" #@param ['strip', 'boxplot']
include_microarray_data = True #@param {type:'boolean'}

xpress.plot_gene_expression(
    analysis=analysis,
    gene_id=GeneID, 
    microarray=include_microarray_data,
    title="", 
    plot_type=plot_type, 
    height=300)

We can provide a list of genes, and sort by either AGAP identifier, or by the median fold-change. We can also filter out values that are not significant based on a `pvalue_filter` parameter. 

In [ ]:
xpress.plot_gene_expression(
    analysis="gamb_colu_arab_fun", 
    gene_id=["AGAP006222", "AGAP006227", "AGAP006228"], 
    microarray=False,
    sort_by=None,
    pvalue_filter=0.05,
    title="Coeae1f", 
    plot_type='strip', 
    height=300
)

You can also produce a boxplot, although the hovertext doesnt quite work as expected.

In [ ]:
xpress.plot_gene_expression(
    analysis='gamb_colu_arab_fun', 
    gene_id=["AGAP006222", "AGAP006227", "AGAP006228"], 
    sort_by='median', 
    plot_type='boxplot', 
    height=300
    )